In [1]:
%load_ext line_profiler
%load_ext autoreload

%autoreload 2
%reload_ext autoreload

# === IMPORTS ===

import logging, sys
import torch
import seqm
from ase.io import read as ase_read
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
from termcolor import colored


from seqm.seqm_functions.fock import fock
from seqm.seqm_functions.pack import unpack
import seqm.seqm_functions.pack as pack
import torch.nn.functional as F

#=== TORCH OPTIONS ===

torch.set_default_dtype(torch.float64)
# if torch.cuda.is_available():
#     device = torch.device('cuda')
# else:
device = torch.device('cpu')
dtype = torch.float64
# torch.set_printoptions(precision=5, linewidth=200, profile="full", sci_mode=False)
torch.set_printoptions(precision=5, linewidth=200, sci_mode=False, profile = 'short')

Decorating your function! <function KSA_XL_BOMD.one_step at 0x7f7a22ab7d90>


In [4]:
!ls

c2.xyz		  examples	 nodes.html	 requirements.txt
c60.xyz		  h2o.xyz	 __pycache__	 seqm
c6h6.xyz	  isoindigo.xyz  pyseqm.dot	 setup.dot
ch4.xyz		  lib		 pyseqm.dot.png  setup.py
coronene.xyz	  LICENSE	 pyseqm.dot.svg  Test1_SinglePointProp_FNS.png
davidson_0723.py  model.pt	 pyseqm.json	 test_module.py
doc		  My_d_combined  README.md


In [25]:
# colored logging with custom level QM for deeper routines

logging.basicConfig(level=logging.DEBUG,
                    format='%(funcName)s : %(lineno)d : %(levelname)s : %(message)s')

QM1 = evel=logging.DEBUG - 3 # informal level of depth; QM1 - almost always, usually outside of loops
QM2 = evel=logging.DEBUG - 4 #                          QM2 - sometimes, in the loops
QM3 = evel=logging.DEBUG - 5

logging.addLevelName(QM1, "QM1")
def qm1(self, message, *args, **kwargs):
    if self.isEnabledFor(QM1 ):
        self._log(QM1, message, args, **kwargs) 
        
logging.addLevelName(QM2, "QM2")
def qm2(self, message, *args, **kwargs):
    if self.isEnabledFor(QM2):
        self._log(QM2, message, args, **kwargs) 
 
logging.addLevelName(QM3, "QM3")
def qm3(self, message, *args, **kwargs):
    if self.isEnabledFor(QM3 ):
        self._log(QM3, message, args, **kwargs) 
           
        
logging.Logger.qm1 = qm1   
logging.Logger.qm2 = qm2
logging.Logger.qm3 = qm3
  
logger = logging.getLogger()

                              
colors = {'qm'        : ('cyan',     None, None),
          'matrix'    : ('blue',     None, ['bold']),
          'vector'    : ('yellow',   None, ['bold']),
          'evals'     : ('green',    None, ['bold']),
          'warn'     : ('red',    None, ['bold'])
          }

def fmt_log(data, message, fmt):
    """
    fmt_log : formats log message with color and style using termcolor module

    Args:
        data (any): data to print
        message (str or None): message to print, pass None if no message is needed
        fmt (str): style from colors dict

    Returns:
        str: formatted string with color and style
    """    

    if type(data) is list or type(data) is tuple or type(data) is torch.Tensor:
        
        mes = f'{colored(message, colors[fmt][0], colors[fmt][1], attrs=colors[fmt][2])}\n' # add new line to align array
    else:
        mes = f'{colored(message, colors[fmt][0], colors[fmt][1], attrs=colors[fmt][2])} : '
        
    if data == None:
        return mes
    else:
        return mes + str(colored(data, colors[fmt][0], colors[fmt][1], attrs=colors[fmt][2]))

### log

07/13/23 - QM part seems to be wortking fine
full diagonalization agrees with NEXMD
small guess space misses relevant vectors, but large guess includes them


PASCAL 1 COULD BE INCORRECT

### QM routines

In [6]:
def run_seqm_1mol(xyz):
    """
    run_seqm_1mol : run PYSEQM for a single molecule

    Args:
        xyz (str): path to xyz file

    Returns:
        Molecule object: PYSEQM object with molecule data
    """    
    
    atoms = ase_read(xyz)
    species = torch.tensor([atoms.get_atomic_numbers()], dtype=torch.long, device=device)
    coordinates = torch.tensor([atoms.get_positions()], dtype=dtype, device=device)
    
    const = Constants().to(device)

    elements = [0]+sorted(set(species.reshape(-1).tolist()))

    seqm_parameters = {
                    'method' : 'PM3',  # AM1, MNDO, PM#
                    'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                    'scf_converger' : [2,0.0], # converger used for scf loop
                                            # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                            # [1], adaptive mixing
                                            # [2], adaptive mixing, then pulay
                    'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                                #[True, eps] or [False], eps for SP2 conve criteria
                    'elements' : elements, #[0,1,6,8],
                    'learned' : [], # learned parameters name list, e.g ['U_ss']
                    #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                    'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                    'eig' : True,
                    'excited' : True,
                    }

    mol = seqm.Molecule.Molecule(const, seqm_parameters, coordinates, species).to(device)

    ### Create electronic structure driver:
    esdriver = Electronic_Structure(seqm_parameters).to(device)

    ### Run esdriver on m:
    esdriver(mol)
    
    return mol

In [7]:
def form_L_xi(vexp1, molecule, N_cis, N_rpa, CIS = True):
    """
    form_L_xi: build A matrix for CIS
               splits guess density into symmetric and antisymmetric parts
               unclear why returns A @ b (guess vector)
               see NEXMD code for QM details
               #! RPA is not implemented yet
               
    Args:
        vexp1 (tensor): guess vector
        molecule (PYSEQM object): _description_
        N_cis (int): dimension of CIS space, nocch*nvirt
        N_rpa (int): N_cis *2
        CIS (bool, optional): CIS or TDHF (RPA) Defaults to True.

    """        

    m = molecule
    gss = m.parameters['g_ss']
    gsp = m.parameters['g_sp']
    gpp = m.parameters['g_pp']
    gp2 = m.parameters['g_p2']
    hsp = m.parameters['h_sp']
    
    mask  = m.mask
    maskd = m.maskd
    idxi  = m.idxi
    idxj  = m.idxj
    nmol  = m.nmol
    molsize = m.molsize
    w       = m.w
    nHeavy = m.nHeavy
    nHydro = m.nHydro
    
    eta = torch.zeros((N_rpa), device=device) 
    
    #print('vexp1.shape', vexp1.shape)
   # print('eta.shape', eta.shape)
    eta[:vexp1.size(0)] = vexp1 # eta is stored as |X|; dcopy?
    
    eta_orig = torch.clone(eta)
    # print('eta_orig.shape', eta_orig.shape)
    # print('eta_orig\n', eta_orig)
    
   # print('m.C MO', m.C_mo[0])
    # print('eta.shape', eta.shape)
    # print(eta)
    eta_ao =  mo2ao(N_cis, eta, m, full=False)     # mo to ao basis (mo2site)
    # print('eta_ao.shape', eta_ao.shape)
    # print(eta_ao)

    eta_ao_sym, eta_ao_asym = decompose_to_sym_antisym(eta_ao) # decompose to sym and asym

    # Vxi - build 2e integrals in AO basis: G(guess density) in F = H_core + G
    # note density is split into sym and anisym matrices
    # sym is processed as padded 3d array in PYSEQM, see fock module 
    # antisym: 2c-2e works with modified PYSEQM routine; should be antisimmterized afterwards
    # antisym: 1c-2e (diagonal) are taken from NEXMD for now - ugly code with loops
    # TODO: vectorize 1c-2e part
    
    #------------------symmetric------------------------------
    G_sym   =  build_G_sym(eta_ao_sym,
                        gss, gsp, gpp, gp2, hsp,
                        mask, maskd, idxi, idxj, nmol, molsize,
                        w,
                         nHeavy,
                         nHydro)
    
   # print('G_SYM\n', G_sym)
    # G sym is 1c-2e and 2c-2e of symmetric part of guess density
    
    
    
    # pack 2c-2e part to standard shape
    G_sym = pack.pack(G_sym, nHeavy, nHydro)
    # print('G_sym \n', G_sym)
    
    G_tot = build_G_antisym_vect(eta_ao, eta_ao_asym, G_sym,
                            gss, gsp, gpp, gp2, hsp,
                            mask, maskd, idxi, idxj, nmol, molsize,
                            w, 
                            m,
                            nHeavy,
                            nHydro)
    
   # print('G_tot.shape', G_tot.shape)
   # print('G_tot\n', G_tot)                  
    # build_G_antisym returns both sym and antisym!
    # TODO: refactor into: 2c-2e antisym, 1c-2e antisym
    # TODO: vectorize 1c-2e antisym, avoid ugly loops
    #! remember about making 2c-2e diagonal 0

    # print('G total \n', G_tot)
    
    # print('============================================')
    # print('Converting Gao full back into MO basis')
    G_mo = ao2mo(N_cis, N_rpa, m, G_tot[0], m.C_mo, full=False) # G in MO basis #! [0] not batched yet
    # print('G_mo.shape', G_mo.shape)
    # print('G_mo\n', G_mo)
    
    # multiply by MO differencies

    return G_mo


In [7]:
def decompose_to_sym_antisym(A):
    """
    decomposes matrix into symmetric and antisymmetric parts

    Args:
        A (tensor): some matrix

    Returns:
        tuple of tensors: sym and antisym parts
    """   

    A_sym = 0.5 * (A + A.T)
    A_antisym = 0.5 * (A - A.T)
    
    return A_sym, A_antisym

In [8]:
def mult_by_mo_e(N_cis, mol, G_mo, eta_orig):
    
    # print('eta ORIG SLOW', eta_orig)
    m = mol
    ii=0
    # kron = []
    for p in range(m.nocc):
        # print('p', p)
        for h in range(m.nocc, m.norb):
            # print('h', h)
            # print('i', i)
            f = m.e_mo[0][h] - m.e_mo[0][p]
            # print('f SLOW', f)
            G_mo[ii] = G_mo[ii] + f * eta_orig[ii]
            # kron.append(f * eta_orig[ii])
            # G_mo[ii+N_cis] = -G_mo[ii + N_cis] + f * eta_orig[ii+N_cis]
            ii += 1
    # print('kron', kron)
    # print('G_mo AFTER SLOW', G_mo)
    return G_mo

In [10]:
#! seems that G_mo is a column of A matrix

In [9]:
def mult_by_mo_e_vect(N_cis, mol, G_mo, eta_orig):
    
    # print('eta ORIG FASR', eta_orig)
    # print('G_mo\n', G_mo)
    m = mol


    occ_idx = torch.arange(int(m.nocc))
    virt_idx = torch.arange(int(m.nocc), int(m.norb))

    # print('occ_idx\n', occ_idx)
    # print('virt_idx\n', virt_idx)
    
    combined_idx = torch.cartesian_prod(occ_idx, virt_idx)  # combinations similar to itertools
    # print('combined_idx\n', combined_idx)
   # print(combined_idx[:, 1])
    
    mo_diff = m.e_mo[0][combined_idx[:, 1]] - m.e_mo[0][combined_idx[:, 0]]  # difference between virtual and occupied
    # print('mo diff', mo_diff)                                                                         # see how elements of A matrix are defined in any CIS paper
                                                                             # Aia,jb=δijδab(ϵa−ϵi)+⟨aj||ib⟩
    mo_kronecker = torch.zeros((N_cis * 2), dtype=torch.float64)
    mo_kronecker[:N_cis] = (mo_diff * eta_orig) 
    

    G_mo += mo_kronecker 
    # print('mo_kronecker\n', mo_kronecker)
    # print('G_mo AFTER FAST', G_mo)
    return G_mo

In [12]:
def build_G_sym(M_ao,
                gss, gsp, gpp, gp2, hsp,
                mask, maskd, idxi, idxj, nmol, molsize,
                w,
                nHydro,
                nHeavy):
    
    
      F = torch.zeros((nmol*molsize**2,4,4), device=device) # 0 Fock matrix to fill
      # # TODO: feed params programmatically
      
      P0 = unpack(M_ao, nHydro, nHeavy, (nHeavy+nHydro)*4) # 
      P0 = torch.unsqueeze(P0, 0) # add dimension
      
      # print('P0.shape', P0.shape)
      # print('P0\n', P0)
      #---------------fill diagonal 1c-2e -------------------
      P = P0.reshape((nmol,molsize,4,molsize,4)) \
          .transpose(2,3).reshape(nmol*molsize*molsize,4,4)
          
      # print('P.shape', P.shape)
      # print('P\n', P)
      
      Pptot = P[...,1,1]+P[...,2,2]+P[...,3,3]
      ## http://openmopac.net/manual/1c2e.html
    #  (s,s)
      TMP = torch.zeros_like(F)
      TMP[maskd,0,0] = 0.5*P[maskd,0,0]*gss + Pptot[maskd]*(gsp-0.5*hsp)
      for i in range(1,4):
          #(p,p)
          TMP[maskd,i,i] = P[maskd,0,0]*(gsp-0.5*hsp) + 0.5*P[maskd,i,i]*gpp \
                          + (Pptot[maskd] - P[maskd,i,i]) * (1.25*gp2-0.25*gpp)
          #(s,p) = (p,s) upper triangle
          TMP[maskd,0,i] = P[maskd,0,i]*(1.5*hsp - 0.5*gsp)
      #(p,p*)
      for i,j in [(1,2),(1,3),(2,3)]:
          TMP[maskd,i,j] = P[maskd,i,j]* (0.75*gpp - 1.25*gp2)

      F.add_(TMP)
      
           
      #-----------------fill 2c-2e integrals----------------
      weight = torch.tensor([1.0,
                        2.0, 1.0,
                        2.0, 2.0, 1.0,
                        2.0, 2.0, 2.0, 1.0],dtype=dtype, device=device).reshape((-1,10))
      
      PA = (P[maskd[idxi]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,10,1))
      PB = (P[maskd[idxj]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,1,10))
      suma = torch.sum(PA*w,dim=1)
      sumb = torch.sum(PB*w,dim=2)
      sumA = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      sumB = torch.zeros_like(sumA)
      
      sumA[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = suma
      sumB[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = sumb
      F.index_add_(0,maskd[idxi],sumB)
      #\sum_A
      F.index_add_(0,maskd[idxj],sumA)
      
      sum = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      # (ss ), (px s), (px px), (py s), (py px), (py py), (pz s), (pz px), (pz py), (pz pz)
      #   0,     1         2       3       4         5       6      7         8        9

      ind = torch.tensor([[0,1,3,6],
                          [1,2,4,7],
                          [3,4,5,8],
                          [6,7,8,9]],dtype=torch.int64, device=device)
      
      Pp = -0.5*P[mask]
      for i in range(4):
        for j in range(4):
            #\sum_{nu \in A} \sum_{sigma \in B} P_{nu, sigma} * (mu nu, lambda, sigma)
            sum[...,i,j] = torch.sum(Pp*w[...,ind[i],:][...,:,ind[j]],dim=(1,2))
      #print('mask', mask)    #! DIFFERS FROM PYSEQM, PROBABLY packing
      F.index_add_(0,mask,sum)

      F0 = F.reshape(nmol,molsize,molsize,4,4) \
             .transpose(2,3) \
             .reshape(nmol, 4*molsize, 4*molsize)
    #
      F0.add_(F0.triu(1).transpose(1,2))     
      
      F0 = 2 * F0 #! BE CAREFUL
      # print('F0.shape', F0.shape)
      # print(F0) 
      
      return F0

In [10]:
def build_G_antisym_vect(eta_ao, eta_ao_asym, G_sym,
                gss, gsp, gpp, gp2, hsp,
                mask, maskd, idxi, idxj, nmol, molsize,
                w, 
                m,
                nHydro,
                nHeavy):

      # TODO; figure how/why constants are defined in fock_skew
      #!
      #! CHECK
      #!
      
      # print('G SYM\n', G_sym)
      
      eta_anti = torch.zeros((m.norb*(m.norb+1)//2), device=device)
      indices = torch.tril_indices(int(m.norb), int(m.norb), offset = 0)
      eta_anti = 0.5 * (eta_ao[indices[0], indices[1]] - eta_ao[indices[1], indices[0]])
      eta_anti_2d = torch.zeros((m.norb, m.norb), device='cpu') 
      eta_anti_2d[indices[1], indices[0]] = -eta_anti 
      eta_anti_2d = eta_anti_2d - eta_anti_2d.T
      
      # ===== VECTORIZED ============
      F = torch.zeros((nmol*molsize**2,4,4), device=device) # 0 Fock matrix to fill
      # # TODO: feed params programmatically
      
      P0 = unpack(eta_anti_2d, nHydro, nHeavy, (nHeavy+nHydro)*4) # 
    #   print('P0.shape', P0.shape)
    #   print('P0\n', P0)
      
      P0 = torch.unsqueeze(P0, 0) # add dimension
      
      # print('P0.shape', P0.shape)
      # print('P0\n', P0)
      #---------------fill diagonal 1c-2e -------------------
      P = P0.reshape((nmol,molsize,4,molsize,4)) \
          .transpose(2,3).reshape(nmol*molsize*molsize,4,4)
          
      Pptot = P[...,1,1]+P[...,2,2]+P[...,3,3]
      
      TMP = torch.zeros_like(F)
      
      #! MODIFIED BY FNS
      for i in range(1,4):
          #(p,p)
          # ! TWO LINES BELOW COULD BE NEEDED, NOT SURE
          # TMP[maskd,i,i] = P[maskd,0,0]*(gsp-hsp) + \
          #                 + (Pptot[maskd] - P[maskd,i,i]) * (0.6*1.25*gp2-0.25*gpp)
        #   (s,p) = (p,s) upper triangle
          # TMP[maskd,i,i] = 100
          TMP[maskd,0,i] = P[maskd,0,i]*(hsp - gsp)
      #(p,p*)
      for i,j in [(1,2),(1,3),(2,3)]:
          TMP[maskd,i,j] = 2*P[maskd,i,j]* (0.25*gpp - 0.6*1.25*gp2)

    #   print('*** TMP *** \n', TMP)
     # ! MAYBE SHOULD be ANTISYMMETRIZD TMP = 
      # TMP = 0.5* ( TMP - TMP.T)
      # F.add_(TMP)  
          
      TMP = TMP.reshape(nmol,molsize,molsize,4,4) \
             .transpose(2,3) \
             .reshape(nmol, 4*molsize, 4*molsize)
             
      TMP = pack.pack(TMP, m.nHeavy, m.nHydro)
      TMP = TMP - TMP.transpose(1,2)
      # print('TMP.shape', TMP.shape)
      # print('TMP\n', TMP)

      # build 2c-2e part of antisymmetric G
      # copied from FOCK
      
      #P = P[...,1,1]+P[...,2,2]+P[...,3,3] #! MODIFIED
      #-----------------fill 2c-2e integrals----------------
      weight = torch.tensor([1.0,
                        2.0, 1.0,
                        2.0, 2.0, 1.0,
                        2.0, 2.0, 2.0, 1.0],dtype=dtype, device=device).reshape((-1,10))
      
      PA = (P[maskd[idxi]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,10,1))
      PB = (P[maskd[idxj]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,1,10))
      suma = torch.sum(PA*w,dim=1)
      sumb = torch.sum(PB*w,dim=2)
      sumA = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      sumB = torch.zeros_like(sumA)
      
      sumA[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = suma
      sumB[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = sumb
      F.index_add_(0,maskd[idxi],sumB)
      #\sum_A
      F.index_add_(0,maskd[idxj],sumA)
      
      sum = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      # (ss ), (px s), (px px), (py s), (py px), (py py), (pz s), (pz px), (pz py), (pz pz)
      #   0,     1         2       3       4         5       6      7         8        9

      ind = torch.tensor([[0,1,3,6],
                          [1,2,4,7],
                          [3,4,5,8],
                          [6,7,8,9]],dtype=torch.int64, device=device)
      
      Pp = -0.5*P[mask]
      for i in range(4):
        for j in range(4):
            #\sum_{nu \in A} \sum_{sigma \in B} P_{nu, sigma} * (mu nu, lambda, sigma)
            sum[...,i,j] = torch.sum(Pp*w[...,ind[i],:][...,:,ind[j]],dim=(1,2))
     # print('mask', mask)    #! DIFFERS FROM PYSEQM, PROBABLY packing
      F.index_add_(0,mask,sum)

      F0 = F.reshape(nmol,molsize,molsize,4,4) \
             .transpose(2,3) \
             .reshape(nmol, 4*molsize, 4*molsize)
    #


      F0.add_(F0.triu(1).transpose(1,2))     
      
      rows, cols = torch.tril_indices(F0.shape[1], F0.shape[2])
      F0[0][rows, cols] *= -1
      F0[0][torch.eye(F0.shape[1]).bool()] *= -1
      
      # F0 is still symmetric, probably symmetrized above
      # here we make it antisymmetric back
    #   F0 = 2 * F0 
      

       #! BE WARNED, THIS iS TAKEN FROM OLD NEXMD, PYSEQM produces non-zero diagonal
    #   F0 = F0
    #  print('G ANTISYM shape', F0.shape)
    #  print('G ANTISYM\n', F0*2)
    
    # BEFORE FINAL ASSEMBLY
     
      F0 = pack.pack(F0, m.nHeavy, m.nHydro)
      F0[0].diagonal().fill_(0)
      
      # print('G ANTISYM\n', F0*2)
      G_sym = torch.unsqueeze(G_sym, 0)
      G_full = G_sym +  F0*2 + TMP# summ of G_sym(sym 1c2e + 2c2e) + F0 (antisym 1c2e + 2c2e) 
      # print('G_full shape', G_full.shape)
      # print('G_full\n', G_full)
      # print('G_full\n', G_full)
      return G_full[0] # ! WORKS FOR ONE ONLY

### AUX routines

In [11]:
def orthogonalize_torch(U, eps=1e-15):
    """
    Orthogonalizes the matrix U (d x n) using Gram-Schmidt Orthogonalization.
    If the columns of U are linearly dependent with rank(U) = r, the last n-r columns 
    will be 0.
    
    Args:
        U (numpy.array): A d x n matrix with columns that need to be orthogonalized.
        eps (float): Threshold value below which numbers are regarded as 0 (default=1e-15).
    
    Returns:
        (numpy.array): A d x n orthogonal matrix. If the input matrix U's cols were
            not linearly independent, then the last n-r cols are zeros.
    
    Examples:
    ```python
    >>> import numpy as np
    >>> import gram_schmidt as gs
    >>> gs.orthogonalize(np.array([[10., 3.], [7., 8.]]))
    array([[ 0.81923192, -0.57346234],
       [ 0.57346234,  0.81923192]])
    >>> gs.orthogonalize(np.array([[10., 3., 4., 8.], [7., 8., 6., 1.]]))
    array([[ 0.81923192 -0.57346234  0.          0.        ]
       [ 0.57346234  0.81923192  0.          0.        ]])
    ```
    """
    
    n = len(U[0])
    # numpy can readily reference rows using indices, but referencing full rows is a little
    # dirty. So, work with transpose(U)
    V = U.T
    for i in range(n):
        prev_basis = V[0:i]     # orthonormal basis before V[i]
        coeff_vec = prev_basis @ V[i].T  # each entry is np.dot(V[j], V[i]) for all j < i
        # subtract projections of V[i] onto already determined basis V[0:i]
        V[i] -= (coeff_vec @ prev_basis).T
        if torch.norm(V[i]) < eps:
            V[i][V[i] < eps] = 0.   # set the small entries to 0
        else:
            V[i] /= torch.norm(V[i])
    return V.T

In [12]:
def gen_V(N_cis, N_rpa, n_V_start):
    
    # returns vexp1 - guess vector for L-xi routine
    logger.qm2(fmt_log(n_V_start, 'n_V_start', 'qm'))
    rrwork = torch.zeros(N_rpa * 4, device=device)
    i = 0
    for ip in range(mol.nocc):
        for ih in range(mol.nvirt):
            rrwork[i] = mol.e_mo[0][mol.nocc + ih] - mol.e_mo[0][ip]  # !Lancos vectors(i) ???
            i += 1                                                                      #  TODO: [0] should be replaced by m batch index
                                                                                    
    rrwork_sorted, indices = torch.sort(rrwork[:N_cis], descending=False, stable=True) # preserve order of degenerate
    logger.qm2(fmt_log(rrwork_sorted, 'rrwork_sorted', 'qm'))


    # vexp1 = torch.zeros((N_cis, N_cis), device=device)
    vexp1 = torch.zeros((N_cis, N_cis), device=device)
    
    row_idx = torch.arange(0, int(N_cis), device=device)
    col_idx = indices[:N_cis]

    vexp1[row_idx, col_idx] = 1.0 
    logger.qm2(fmt_log(vexp1, 'V  BEFORE SELECTING PART', 'qm'))
    logger.qm2(fmt_log(vexp1.shape, 'V shape', 'qm'))
   #! THIS IS NEW, TAKE only part 

    V = vexp1[:,  :n_V_start]
    logger.qm2(fmt_log(V, 'V = vexp1', 'qm'))
    logger.qm2(fmt_log(V.shape, 'V shape', 'qm'))

    return V

# TODO: check whether L_xi should be regenerated during expansion each time or just part of it

### DAVIDSON routines

In [13]:
logger.setLevel(logging.DEBUG)  # custom logging level; lower than DEBUG
                               # printed above QM (QM, DEBUG, INFO, etc)

In [14]:
def davidson(mol, N_exc, keep_n, n_V_max,  max_iter, tol):
    """
    Davidson algorithm for solving eigenvalue problem of large sparse diagonally dominant matrices
    Hamiltonian is not generated or stored explicitly, only matrix-vector products are used on-the fly:
    guess space V should be orthogonalized at each iteration
    M (projection of smaller size) is V.T @ H @ V 
    #! RPA (TDHF) is not implemented yet, non-Hermitian (non-symmetric), requires also left eigenvectors 
    note that notation differes between implementations: V.T x A x V is bAb
    # TODO: 1) check if convergence of e_vals is needed
    # TODO: 2) vectorize and optimize orthogonalization
    # TODO: 3) check if some vectors should be dropped 
    # TODO: 4) eliminate loops 
    # TODO: 5) check if whole M should be regenerated, or only sub-blocks corresponding to new guess vectors
    # TODO: 6) add parameter checker like Krylov dims << N_cis

    Args:
        mol (PYSEQM object): object to hold all qm data from PYSEQM
        N_exc (int)        : number of excited states to calculate
        keep_n (int)       : number of e_vals, e_vecs to keep at each iteration
        n_V_max (int)      : maximum size of Krylov subspace, 
                             projected matrix will be no more than M(n_V_max x n_V_max)
        max_iter (int)     : maximum number of iterations in Davidson
        tol (float)        : treshold for residual
        
    Returns:
        tuple of tensors: eigenvalues (excitation energies in default units, eV) and eigenvectors 
    """    
    
    n_V_start = N_exc * 2 # dimension of Krylov subspace, analogue of nd1  
    N_cis = mol.nocc * mol.nvirt
    N_rpa = 2 * N_cis
    
    term = False  # terminate algorithm
    iter = 0
    L_xi = torch.zeros((N_rpa, n_V_start), device=device)
    L_xi2 = torch.zeros((N_rpa, n_V_start), device=device)
    V = gen_V(N_cis, N_rpa, n_V_start) # generate initial guess, V here #! should be renamed
    diag = None # create diagonal of M only once
    
    while iter < max_iter and not term: # Davidson loop
        
        if iter > 0: # skip first step, as initial V is orthogonal
            V = orthogonalize_torch(V)
            
        print('=================================', flush=True)
        print(colored(f' ITERATION : {iter} ', 'red', 'on_white', attrs=['bold']), flush=True)
        print('SUBSPACE SIZE V: ', V.shape, flush=True)
        print('=================================')
       
        # ---------- form A x b product --------------------
        L_xi = torch.zeros((N_rpa, V.shape[1] ), device=device) #! NOT iter here
        logger.qm1(fmt_log(V, 'V BEFORE L_xi after ORTO', 'qm'))
        for i in range(V.shape[1]): 
            # print('=================================', flush=True)
            logger.qm3('Lxi iterations=%s', i)
            L_xi[:,i] = form_L_xi(V[:,i], mol, N_cis, N_rpa)
            # L_xi2[:,i] = torch.clone(L_xi[:,i])
            
            # L_xi[:,i]  = mult_by_mo_e(N_cis, mol, L_xi[:,i], V[:,i]) 
            L_xi[:,i] = mult_by_mo_e_vect(N_cis, mol, L_xi[:,i], V[:,i]) 
            # multiply by MO energies
            
            
            
            # raise ValueError('STOP')
            logger.qm3(fmt_log(L_xi[:,i], 'L_xi[:,i]', 'qm'))
        
        L_xi[N_cis:, :] = L_xi[:N_cis] #! TODO: make sure that this A+B, A-B, not just copy for RPA
    
        right_V = L_xi[N_cis:] # (A)b 
        
        logger.qm1(fmt_log(right_V.shape, 'right_V shape', 'matrix'))
        logger.qm1(fmt_log(right_V, 'right_V', 'matrix'))       
        # ---------- form b.T x Ab product --------------------
        
        M =  V.T @ right_V
        
        # logger.debug(fmt_log(M.shape, 'M shape', 'qm'))
        # logger.debug(fmt_log(M, 'M', 'qm'))
        if iter == 0:
            diag = torch.diag(M) # create diagonal only once
            
        iter += 1
        
        logger.qm1(fmt_log(diag, 'diag', 'qm'))
    
        # ---------- diagonalize projection M --------------------
        r_eval, r_evec = torch.linalg.eigh(M) # find eigenvalues and eigenvectors
       
        r_eval = r_eval.real
        r_evec = r_evec.real
        r_eval, r_idx = torch.sort(r_eval, descending=False) # sort eigenvalues in ascending order
        logger.debug(fmt_log(r_eval, 'RIGHT EVALS', 'evals'))
        r_evec = r_evec[:, r_idx] # sort eigenvectors accordingly
    
        e_val_n = r_eval[:keep_n] # keep only the lowest keep_n eigenvalues; full are still stored as e_val
        e_vec_n = r_evec[:, :keep_n]
        resids = torch.zeros(V.shape[0], len(e_val_n)) # account for left and right evecs

        # ---------- calculate residual vectors --------------------
        for j in range(len(e_val_n)): # calc residuals 
            resids[:,j] = right_V @ e_vec_n[:,j] - e_val_n[j] * (V @ e_vec_n[:,j])
            
       # logger.debug(fmt_log(resids, 'resids', 'matrix'))     
        resids_norms_r = torch.tensor([resids[:,x].norm() for x in range(resids.shape[1])])

        # ---------- expand guess space V buy not-converged resids --------------------
        # !!! PROBABLY HIGHLY INEFFICIENT !!! 
        if torch.any(resids_norms_r > tol):
            mask_r = resids_norms_r >= tol
            large_res_r = resids[:,mask_r] # residuals larger than tol
           # logger.debug(fmt_log(large_res_r, 'LARGE RESIDUALS', 'vector'))           
            large_res_r.to(device)
            cor_e_val_r = e_val_n[mask_r] # corresponding eigenvalues !!! check if matches
            
            # ------keep adding new resids --------------------
            if V.shape[1] <= n_V_max:     

                    for j in range(large_res_r.shape[1]):
                        if V.shape[1] <= n_V_max:
                            s = large_res_r[:,j] # conditioned residuals > tol

                            if s.norm() >= tol:
                                logger.debug(fmt_log((s.norm().item()), 'NORM of RESIDUAL', 'warn'))
                                denom = (diag[j] - cor_e_val_r[j])
                                denom.to(device) 
                                s = s/denom # conditioned residuals
                                s.to(device)
                                # logger.debug(fmt_log(s.norm(), 'NORM OF NEW RESIDUAAL', 'vector'))
                                V = torch.column_stack((V, s/s.norm()))
                            else:
                                pass
            # ------ collapse (restart) if space V is too large; mix eigenvectors with V------------
            else:
                logger.debug(fmt_log(None, '!!!! MAX subspace reached !!!!', 'warn'))
                #logger.debug(fmt_log(V, 'V before collapse', 'qm'))

                V =  V @ r_evec[:, :n_V_start]
                logger.debug(fmt_log(V.shape, 'V shape after restart', 'qm'))
                #logger.debug(fmt_log(V, 'V AFTER collapse', 'qm'))

                continue

        else:
            term = True
            print('============================', flush=True)
            print('all residuals are below tolerance')
            print('DAVIDSON ALGORITHM CONVERGED', flush=True)
            print('============================', flush=True)

            return r_eval, r_evec

    # runs after big loop if did not converge
    print('============================', flush=True)
    print('!!! DAVIDSON ALGORITHM DID NOT CONVERGE !!!', flush=True)
    print('============================', flush=True)
    
    return r_eval, r_evec

In [15]:
# mol = run_seqm_1mol('c6h6.xyz')
# eval, _ = davidson(mol = mol, 
#                    N_exc = 8,
#                    keep_n = 4,
#                    n_V_max = 50, 
#                    max_iter = 50, 
#                    tol = 1e-6)

# logger.debug(fmt_log(eval, 'FINAL eval ', 'evals'))



mol = run_seqm_1mol('h2o.xyz')
eval, _ = davidson(mol = mol, 
                   N_exc = 3,
                   keep_n = 2,
                   n_V_max = 10, 
                   max_iter = 3, 
                   tol = 1e-6)

logger.debug(fmt_log(eval, 'FINAL eval ', 'evals'))

 ITERATION : 0 
SUBSPACE SIZE V:  torch.Size([8, 6])


/tmp/ipykernel_256895/1247529461.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1678411187366/work/torch/csrc/utils/tensor_new.cpp:245.)
  species = torch.tensor([atoms.get_atomic_numbers()], dtype=torch.long, device=device)


NameError: name 'form_L_xi' is not defined

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [24]:
mol = run_seqm_1mol('c60.xyz')
%lprun -u 1 -f davidson davidson(mol = mol, N_exc = 8, keep_n = 4, n_V_max = 60, max_iter = 10, tol = 1e-6)
# %lprun -u 1 -f davidson davidson(mol = mol, N_exc = 8,keep_n = 4, n_V_max = 60, max_iter = 1, tol = 1e-6)

 ITERATION : 0 
SUBSPACE SIZE V:  torch.Size([14400, 16])


davidson : 93 : DEBUG : RIGHT EVALS
tensor([ 2.69621,  3.59425,  3.67799,  5.22364,  5.24623,  5.35856,  5.39645,  5.42210,  8.16187,  8.18378,  8.20683,  9.60633,  9.64560,  9.68289, 10.07326, 10.08923], grad_fn=<SortBackward0>)
davidson : 124 : DEBUG : NORM of RESIDUAL : 1.1071475502570793
davidson : 124 : DEBUG : NORM of RESIDUAL : 1.1745613697839479
davidson : 124 : DEBUG : NORM of RESIDUAL : 1.2440393568512984
davidson : 124 : DEBUG : NORM of RESIDUAL : 0.982610381588808


 ITERATION : 1 
SUBSPACE SIZE V:  torch.Size([14400, 20])


/tmp/ipykernel_165479/3498202963.py:34: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/TensorShape.cpp:3571.)
  coeff_vec = prev_basis @ V[i].T  # each entry is np.dot(V[j], V[i]) for all j < i


davidson : 93 : DEBUG : RIGHT EVALS
tensor([ 2.38429,  3.28952,  3.36376,  4.84494,  5.22684,  5.35821,  5.39267,  5.41620,  6.64028,  7.67619,  8.16193,  8.16411,  8.18378,  8.20718,  8.71632,  9.60640,  9.64560,  9.68289, 10.07326,
        10.08923], grad_fn=<SortBackward0>)
davidson : 124 : DEBUG : NORM of RESIDUAL : 1.1906089471926826
davidson : 124 : DEBUG : NORM of RESIDUAL : 1.2281865635237594
davidson : 124 : DEBUG : NORM of RESIDUAL : 1.2090488227453984
davidson : 124 : DEBUG : NORM of RESIDUAL : 1.5146511439033261


...

davidson : 93 : DEBUG : RIGHT EVALS
tensor([ 2.07258,  2.57547,  2.61908,  2.95683,  2.97863,  4.00436,  4.05336,  4.81522,  4.96735,  5.20605,  5.35774,  5.40238,  5.44246,  6.17226,  6.46735,  6.50312,  8.16186,  8.18360,  8.20712,
         9.60655,  9.64558,  9.68286,  9.73826, 10.07325, 10.08904, 10.28883, 11.37178, 11.44548, 15.16452, 15.58752, 15.65265, 16.37688, 22.29071, 23.09001, 23.12903, 23.41535, 32.77689, 33.15135,
        33.30378, 33.77109, 41.95614, 42.18481, 42.34661, 42.51215, 48.02438, 48.23598, 48.68313, 48.68746], grad_fn=<SortBackward0>)
davidson : 124 : DEBUG : NORM of RESIDUAL : 0.5293483447808045
davidson : 124 : DEBUG : NORM of RESIDUAL : 0.40376026886459065
davidson : 124 : DEBUG : NORM of RESIDUAL : 0.37093669952497604
davidson : 124 : DEBUG : NORM of RESIDUAL : 1.0606328282889572


 ITERATION : 9 
SUBSPACE SIZE V:  torch.Size([14400, 52])


davidson : 93 : DEBUG : RIGHT EVALS
tensor([ 2.04969,  2.56195,  2.60784,  2.77427,  2.86856,  3.93064,  4.00799,  4.68235,  4.78935,  5.15930,  5.35717,  5.40064,  5.43201,  5.49642,  5.92272,  5.96509,  7.79235,  8.12436,  8.16187,
         8.19638,  8.23234,  9.18254,  9.29289,  9.60732,  9.64558,  9.68287, 10.07326, 10.08923, 13.24372, 13.41962, 13.75118, 13.79083, 19.59087, 19.76981, 19.82959, 20.60179, 27.75684, 28.08043,
        28.41900, 28.66074, 36.81289, 37.04319, 37.31365, 37.52671, 43.91682, 44.14594, 44.42066, 44.46645, 48.87550, 49.15220, 49.45905, 49.60140], grad_fn=<SortBackward0>)
davidson : 124 : DEBUG : NORM of RESIDUAL : 0.5120440022477968
davidson : 124 : DEBUG : NORM of RESIDUAL : 0.4035070359452903
davidson : 124 : DEBUG : NORM of RESIDUAL : 0.37068271158424526
davidson : 124 : DEBUG : NORM of RESIDUAL : 1.3950938441362728


!!! DAVIDSON ALGORITHM DID NOT CONVERGE !!!


Timer unit: 1 s

Total time: 7.58392 s
File: /tmp/ipykernel_165479/3418432135.py
Function: davidson at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def davidson(mol, N_exc, keep_n, n_V_max,  max_iter, tol):
     2                                               """
     3                                               Davidson algorithm for solving eigenvalue problem of large sparse diagonally dominant matrices
     4                                               Hamiltonian is not generated or stored explicitly, only matrix-vector products are used on-the fly:
     5                                               guess space V should be orthogonalized at each iteration
     6                                               M (projection of smaller size) is V.T @ H @ V 
     7                                               #! RPA (TDHF) is not implemented yet, non-Hermitian (non-symmetric), requires also left eigenvect